In [ ]:
import os

In [ ]:
os.environ

In [ ]:
migs= 'MIG-c3ce33ce-ced8-5961-bb87-2b40eb100277,MIG-280489c4-1d98-5b07-b4f6-2fc85fc874fa,MIG-c432df19-0894-5232-ac1c-9a3440fc267e,MIG-e8f61a95-352a-56cc-b95d-0c35fc14e8bf'

In [ ]:
env = dict(os.environ)

In [ ]:
env['CUDA_VISIBLE_DEVICES']=migs

In [ ]:
os.environ = env

In [ ]:
os.environ

In [ ]:
import os
import ray

# Set CUDA_VISIBLE_DEVICES to control which GPUs are visible to Ray
#os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6"#'MIG-c3ce33ce-ced8-5961-bb87-2b40eb100277,MIG-280489c4-1d98-5b07-b4f6-2fc85fc874fa,MIG-c432df19-0894-5232-ac1c-9a3440fc267e,MIG-e8f61a95-352a-56cc-b95d-0c35fc14e8bf'#"0,1,2,3,4,5,6"

# Initialize Ray with the appropriate configuration
ray.init()


In [ ]:
resources = ray.cluster_resources()
print(resources)


In [ ]:
import numpy as np
import gymnasium as gym
from gym_CTgraph import CTgraph_env
from gym_CTgraph.CTgraph_plot import CTgraph_plot
from gym_CTgraph.CTgraph_conf import CTgraph_conf
from gym_CTgraph.CTgraph_images import CTgraph_images
import json
import argparse
#import json
import random
import matplotlib.pyplot as plt
import timeit

from gym.spaces import Box, Discrete, Dict
from ray.tune.registry import register_env
from ray import tune

from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import argparse
#import gym
from gym.spaces import Discrete, Box
import numpy as np
#import os
import random

import ray
from ray import tune
from ray.rllib.agents import ppo
from ray.rllib.env.env_context import EnvContext
from ray.rllib.models import ModelCatalog
from ray.rllib.models.tf.tf_modelv2 import TFModelV2
from ray.rllib.models.tf.fcnet import FullyConnectedNetwork
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.models.torch.fcnet import FullyConnectedNetwork as TorchFC
from ray.rllib.utils.framework import try_import_tf, try_import_torch
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.tune.logger import pretty_print

In [ ]:
'''import ray
ray.init()
resources = ray.cluster_resources()
print(resources)'''


In [ ]:
from ray import tune

In [ ]:
def env_creator(env_config={}):
    # fetch the parameters from the json file
    configuration = CTgraph_conf("C:\CT-Graph_Native\CT-graph\CT_Graph_Christos\graph09.json")
    conf_data = configuration.getParameters()
    # print configration data
    print(json.dumps(conf_data, indent=3))
    #env = gym.make('CTgraph-v0')
    # instantiate the maze
    #start = timeit.timeit()
    #env = gym.make('CTgraph-v0')
    #end = timeit.timeit()
    #print(end - start)

    imageDataset = CTgraph_images(conf_data)
    env = gym.make('CTgraph-v0', conf_data=conf_data, images=imageDataset)
    env.reset()

    # get a random path from the maze
    #high_reward_path = np.array(np.random.randint(1, conf_data['graph_shape']['branching_factor'], size=conf_data['graph_shape']['depth']))
    # use this random path to set the path to the high reward. Note that the maze would have already a high_reward_path from the initialisation
    #env.set_high_reward_path(high_reward_path)
    #seed_everything(env)  
    
    return env

In [ ]:
register_env("CT-Graph_d2_b2_wp05_crv0_MDP", env_creator)

In [ ]:
tune.run("IMPALA",
             # algorithm specific configuration
             config={"env": "CT-Graph_d2_b2_wp05_crv0_MDP",  #
                     "framework": "tf2",
                     "seed":958,
                     #"num_gpus": 1,
                     "num_rollout_workers": 2,
                     "num_envs_per_worker": 1,
                     "num_gpus_per_worker": 0,
                     "train_batch_size": 512,
                     #"use_critic":True,
                     #"use_gae": True,
                     #"lambda": 0.99,
                     #"sgd_minibatch_size": 16,
                     "num_sgd_iter": 8,
                     "shuffle_sequences": False,
                     "entropy_coeff": 0.00015,
                     #"clip_param": 0.1,
                     "grad_clip": 5,
                     "lr": 0.00015,
                     #"num_cpus_per_worker": 1,
                     #"model": {"custom_model": "pa_model", },
                     "evaluation_interval": 1,
                     "evaluation_num_episodes": 2
                     },
             local_dir="CT-Graph_d2_b2_wp05_crv0_MDP06",  # directory to save results
             checkpoint_freq=2,  # frequency between checkpoints
             keep_checkpoints_num=6,
             stop={"timesteps_total": 1500000}
             )
